In [1]:
import open3d as o3d
import numpy as np
import json
import copy

In [2]:
pc1_path = "../labelCloud/pointclouds/cloud_scene5.ply"
pc2_path = "../labelCloud/pointclouds/cloud_scene7.ply"
pc1 = o3d.io.read_point_cloud(pc1_path)
pc2 = o3d.io.read_point_cloud(pc2_path)

In [3]:
def preprocess_point_cloud(pcd, voxel_size):
    print(":: Downsample with a voxel size %.3f." % voxel_size)
    pcd_down = pcd.voxel_down_sample(voxel_size)

    radius_normal = voxel_size * 2
    print(":: Estimate normal with search radius %.3f." % radius_normal)
    pcd_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

    radius_feature = voxel_size * 5
    print(":: Compute FPFH feature with search radius %.3f." % radius_feature)
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    return pcd_down, pcd_fpfh

In [4]:
def prepare_dataset(source, target, voxel_size):
    print(":: Load two point clouds and disturb initial pose.")
    trans_init = np.asarray([[0.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, 0.0],
                             [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0]])
    source.transform(trans_init)

    source_down, source_fpfh = preprocess_point_cloud(source, voxel_size)
    target_down, target_fpfh = preprocess_point_cloud(target, voxel_size)
    return source, target, source_down, target_down, source_fpfh, target_fpfh

In [5]:
def execute_global_registration(source_down, target_down, source_fpfh,
                                target_fpfh, voxel_size):
    distance_threshold = voxel_size * 1.5
    print(":: RANSAC registration on downsampled point clouds.")
    print("   Since the downsampling voxel size is %.3f," % voxel_size)
    print("   we use a liberal distance threshold %.3f." % distance_threshold)
    result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh, True,
        distance_threshold,
        o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
        3, [
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(
                0.9),
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(
                distance_threshold)
        ], o3d.pipelines.registration.RANSACConvergenceCriteria(100000, 0.999))
    return result

In [6]:
source, target, source_down, target_down, source_fpfh, target_fpfh = prepare_dataset(pc1, pc2, voxel_size=0.01)
result_ransac = execute_global_registration(source_down, target_down, source_fpfh, target_fpfh, voxel_size=0.01)
trans_init = result_ransac.transformation

:: Load two point clouds and disturb initial pose.
:: Downsample with a voxel size 0.010.
:: Estimate normal with search radius 0.020.
:: Compute FPFH feature with search radius 0.050.
:: Downsample with a voxel size 0.010.
:: Estimate normal with search radius 0.020.
:: Compute FPFH feature with search radius 0.050.
:: RANSAC registration on downsampled point clouds.
   Since the downsampling voxel size is 0.010,
   we use a liberal distance threshold 0.015.


In [13]:
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    #source_temp.paint_uniform_color([1, 0.706, 0])
    #target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp]),
                                      #zoom=0.4459
                                      #front=[0.9288, -0.2951, -0.2242],
                                      #lookat=[1.6784, 2.0612, 1.4451],
                                      #up=[-0.3402, -0.9189, -0.1996]

In [14]:
reg_p2l = o3d.pipelines.registration.registration_icp(
    pc1, pc2, 0.2, trans_init,
    o3d.pipelines.registration.TransformationEstimationPointToPlane())

In [15]:
draw_registration_result(pc1, pc2, reg_p2l.transformation)